## Add GPS Metadata capture for point, line, polygon layers

This notebook configures an existing point, line or polygon feature layer to record GPS metadata using ArcGIS Field Maps.

The notebook performs the following actions:

- Adds the necessary domains
- Adds a series of fields and associates domains
- Enables attachments if needed (line and polygon layers only)

Note: you must be the owner of the feature layer or an organization admin user

### Steps

1. Update the following variables
    - `org_url`
    - `username`    
    - `item_id`
    - `feature_layer_index`
2. Run all cells


In [ ]:
from arcgis.gis import GIS
from arcgis.features import FeatureLayerCollection

# update this information prior to running
org_url = "https://www.arcgis.com"
username = "doug_nitro"
item_id = "7984f1a3089c47eaad07086b19f6fe92" # item_id of feature layer
feature_layer_index = 0 #index of layer you want to update

gis = GIS(org_url, username)

In [ ]:
# get the feature layer by ID
item = gis.content.get(item_id)

if (item is None):
    raise TypeError("Cannot Find item")

# check the item type is a feature service
if (item.type != 'Feature Service'):
    raise TypeError("Item is not a feature service")

feature_layer_collection = FeatureLayerCollection.fromitem(item)

feature_layer = feature_layer_collection.layers[feature_layer_index]

# get the geometry type
geometry_type = feature_layer.properties.geometryType

# make sure layer is a point, line, or polygon layer
if ((geometry_type != 'esriGeometryPolyline') and 
    (geometry_type != 'esriGeometryPolygon')and 
    (geometry_type != 'esriGeometryPoint')):
    raise TypeError("Feature layer is not a point, line, or polygon layer")

# check and enable attachments
if (geometry_type != 'esriGeometryPoint' and not feature_layer.properties.hasAttachments):    
    feature_layer.manager.update_definition({"hasAttachments": True})
    print ("Enabled attachments")

In [ ]:
# Extract existing fields from Feature layer service definition
feature_layer_fields = feature_layer.properties.fields

# New fields which need to be added
gnss_metadata_fields = {'fields': []}

# Operations list - Add or Update GNSS Metadata fields.
operations = []

# Ensure all fields are present; if not, add them to the definition

if (geometry_type != 'esriGeometryPoint'):

    # esrignss_avg_h_rms
    avg_horizontal_accuracy_field = [field for field in feature_layer_fields if field['name'].lower() == 'esrignss_avg_h_rms']

    if not avg_horizontal_accuracy_field:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_avg_h_rms',
                                                'type': 'esriFieldTypeDouble',
                                                'alias': 'Average horizontal accuracy (m)',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})

    # esrignss_avg_v_rms
    avg_vertical_accuracy_field = [field for field in feature_layer_fields if field['name'].lower() == 'esrignss_avg_v_rms']

    if not avg_vertical_accuracy_field:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_avg_v_rms',
                                                'type': 'esriFieldTypeDouble',
                                                'alias': 'Average vertical accuracy (m)',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})

    # esrignss_worst_h_rms
    worst_horizontal_accuracy_field = [field for field in feature_layer_fields if field['name'].lower() == 'esrignss_worst_h_rms']

    if not worst_horizontal_accuracy_field:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_worst_h_rms',
                                                'type': 'esriFieldTypeDouble',
                                                'alias': 'Worst horizontal accuracy (m)',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})

    # esrignss_worst_v_rms
    worst_vertical_accuracy_field = [field for field in feature_layer_fields if field['name'].lower() == 'esrignss_worst_v_rms']

    if not worst_vertical_accuracy_field:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_worst_v_rms',
                                                'type': 'esriFieldTypeDouble',
                                                'alias': 'Worst vertical accuracy (m)',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})


    # esrignss_worst_fixtype
    worst_fixtype_field = [field for field in feature_layer_fields if field['name'].lower() == 'esrignss_worst_fixtype']

    if worst_fixtype_field:
        # Field does exist check if the domain is set.
        if worst_fixtype_field[0]['domain'] is None:
            #if ([operation for operation in operations if operation == 'updateDefinition']):
            operations.append('updateDefinition')

            worst_fixtype_field_index = feature_layer_fields.index(worst_fixtype_field[0])
            fixtype_domain = {'type': 'codedValue',
                                'name': 'esri_fix_type_domain',
                                'codedValues': [{'name': 'Fix not valid',
                                                'code': 0},
                                                {'name': 'GPS', 'code': 1},
                                                {'name': 'Differential GPS', 'code': 2},
                                                {'name': 'RTK Fixed', 'code': 4},
                                                {'name': 'RTK Float', 'code': 5}]}
            feature_layer_fields[worst_fixtype_field_index]['domain'] = fixtype_domain

    else:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_worst_fixtype',
                                                'type': 'esriFieldTypeSmallInteger',
                                                'alias': 'Worst fix type',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': {'type': 'codedValue',
                                                        'name': 'esri_fix_type_domain',
                                                        'codedValues': [
                                                            {'name': 'Fix not valid', 'code': 0},
                                                            {'name': 'GPS', 'code': 1},
                                                            {'name': 'Differential GPS', 'code': 2},
                                                            {'name': 'RTK Fixed', 'code': 4},
                                                            {'name': 'RTK Float', 'code': 5}]},
                                                'defaultValue': None})

    # esrignss_manual_locations
    num_manual_locations_field = [field for field in feature_layer_fields if field['name'].lower() == 'esrignss_manual_locations']

    if not num_manual_locations_field:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_manual_locations',
                                                'type': 'esriFieldTypeInteger',
                                                'alias': 'Number of manual locations',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})
else:
    # esrignss_positionsourcetype
    positionsourcetype_field = [field for field in feature_layer_fields if field['name'].lower() == 'esrignss_positionsourcetype']

    if positionsourcetype_field:
        # Field does exist; check if the domain is set
        if positionsourcetype_field[0]['domain'] is None:
            operations.append('updateDefinition')

            positionsourcetype_field_index = feature_layer_fields.index(positionsourcetype_field[0])
            positionsourcetype_domain = {'type': 'codedValue',
                                'name': 'esri_positionsourcetype_domain',
                                'codedValues': [{'name': 'Unknown',
                                                'code': 0},
                                                {'name': 'User defined', 'code': 1},
                                                {'name': 'Integrated (System) Location Provider', 'code': 2},
                                                {'name': 'External GNSS Receiver', 'code': 3},
                                                {'name': 'Network Location Provider', 'code': 4},
                                                {'name': 'Snapped', 'code': 5}]},
            feature_layer_fields[positionsourcetype_field_index]['domain'] = positionsourcetype_domain

    else:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_positionsourcetype',
                                                'type': 'esriFieldTypeInteger',
                                                'alias': 'Position source type',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': {'type': 'codedValue',
                                                        'name': 'esri_positionsourcetype_domain',
                                                        'codedValues': [
                                                            {'name': 'Unknown', 'code': 0},
                                                            {'name': 'User defined', 'code': 1},
                                                            {'name': 'Integrated (System) Location Provider', 'code': 2},
                                                            {'name': 'External GNSS Receiver', 'code': 3},
                                                            {'name': 'Network Location Provider', 'code': 4},
                                                            {'name': 'Snapped', 'code': 5}]},
                                                'defaultValue': None})

    # esrignss_receiver
    receiver_field = [field for field in feature_layer_fields if field['name'].lower() == 'esrignss_receiver']
    
    if not receiver_field:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_receiver',
                                                'type': 'esriFieldTypeString',
                                                'alias': 'Receiver name',
                                                'sqlType': 'sqlTypeOther',
                                                'length': 50,
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})

    # esrignss_latitude
    latitude_field = [field for field in feature_layer_fields if field['name'].lower() == 'esrignss_latitude']

    if not latitude_field:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_latitude',
                                                'type': 'esriFieldTypeDouble',
                                                'alias': 'Latitude',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})

    # esrignss_longitude
    longitude_field = [field for field in feature_layer_fields if field['name'].lower() == 'esrignss_longitude']

    if not longitude_field:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_longitude',
                                                'type': 'esriFieldTypeDouble',
                                                'alias': 'Longitude',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})

    # esrignss_altitude
    altitude_field = [field for field in feature_layer_fields if field['name'].lower() == 'esrignss_altitude']

    if not altitude_field:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_altitude',
                                                'type': 'esriFieldTypeDouble',
                                                'alias': 'Altitude',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})

    # esrignss_h_rms
    horizontal_accuracy_field = [field for field in feature_layer_fields if field['name'].lower() == 'esrignss_h_rms']

    if not horizontal_accuracy_field:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_h_rms',
                                                'type': 'esriFieldTypeDouble',
                                                'alias': 'Horizontal accuracy (m)',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})
    # esrignss_v_rms
    vertical_accuracy_field = [field for field in feature_layer_fields if field['name'].lower() == 'esrignss_v_rms']

    if not vertical_accuracy_field:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_v_rms',
                                                'type': 'esriFieldTypeDouble',
                                                'alias': 'Vertical accuracy (m)',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})

    # esrignss_fixdatetime
    fix_time_field = [field for field in feature_layer_fields if field['name'].lower() == 'esrignss_fixdatetime']

    if not fix_time_field:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_fixdatetime',
                                                'type': 'esriFieldTypeDate',
                                                'alias': 'Fix time',
                                                'sqlType': 'sqlTypeOther',
                                                'length': 0,
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})

    # esrignss_fixtype
    fix_type_field = [field for field in feature_layer_fields if field['name'].lower() == 'esrignss_fixtype']

    if fix_type_field:
        # Field does exist; check if the domain is set
        if fix_type_field[0]['domain'] == None:
            if ([operation for operation in operations if operation == 'updateDefinition']):
                operations.append('updateDefinition')

            fix_type_field_index = feature_layer_fields.index(fix_type_field[0])
            fix_type_domain = {'type': 'codedValue',
                                'name': 'esri_fix_type_domain',
                                'codedValues': [{'name': 'Fix not valid',
                                                'code': 0},
                                                {'name': 'GPS', 'code': 1},
                                                {'name': 'Differential GPS', 'code': 2},
                                                {'name': 'RTK Fixed', 'code': 4},
                                                {'name': 'RTK Float', 'code': 5}]}
            feature_layer_fields[fix_type_field_index]['domain'] = fix_type_domain

    else:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_fixtype',
                                                'type': 'esriFieldTypeInteger',
                                                'alias': 'Fix type',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': {'type': 'codedValue',
                                                        'name': 'esri_fix_type_domain',
                                                        'codedValues': [
                                                            {'name': 'Fix not valid', 'code': 0},
                                                            {'name': 'GPS', 'code': 1},
                                                            {'name': 'Differential GPS', 'code': 2},
                                                            {'name': 'RTK Fixed', 'code': 4},
                                                            {'name': 'RTK Float', 'code': 5}]},
                                                'defaultValue': None})

    # esrignss_fixtype
    correction_age_field = [field for field in feature_layer_fields if field['name'].lower() == 'esrignss_correctionage']

    if not correction_age_field:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_correctionage',
                                                'type': 'esriFieldTypeDouble',
                                                'alias': 'Correction age',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})

    # esrignss_stationid
    station_id_field = [field for field in feature_layer_fields if field['name'].lower() == 'esrignss_stationid']

    if not station_id_field:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_stationid',
                                                'type': 'esriFieldTypeInteger',
                                                'alias': 'Station ID',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})

    # esrignss_numsats
    num_sats_field = [field for field in feature_layer_fields if field['name'].lower() == 'esrignss_numsats']

    if not num_sats_field: 
        gnss_metadata_fields['fields'].append({'name': 'esrignss_numsats',
                                                'type': 'esriFieldTypeInteger',
                                                'alias': 'Number of satellites',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})

    # esrignss_pdop
    pdop_field = [field for field in feature_layer_fields if field['name'].lower() == 'esrignss_pdop']

    if not pdop_field:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_pdop',
                                                'type': 'esriFieldTypeDouble',
                                                'alias': 'PDOP',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})

    # esrignss_hdop
    hdop_field = [field for field in feature_layer_fields if field['name'].lower() == 'esrignss_hdop']

    if not hdop_field:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_hdop',
                                                'type': 'esriFieldTypeDouble',
                                                'alias': 'HDOP',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})

    # esrignss_vdop
    vdop_field = [field for field in feature_layer_fields if field['name'].lower() == 'esrignss_vdop']

    if not vdop_field:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_vdop',
                                                'type': 'esriFieldTypeDouble',
                                                'alias': 'VDOP',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})
        
    # esrignss_direction
    direction_field = [field for field in feature_layer_fields if field['name'].lower() == 'esrignss_direction']

    if not direction_field:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_direction',
                                                'type': 'esriFieldTypeDouble',
                                                'alias': 'Direction of travel (°)',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})

    # esrignss_speed
    speed_field = [field for field in feature_layer_fields if field['name'].lower() == 'esrignss_speed']

    if not speed_field:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_speed',
                                                'type': 'esriFieldTypeDouble',
                                                'alias': 'Speed (km/h)',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})

    # esrisnsr_azimuth
    azimuth_field = [field for field in feature_layer_fields if field['name'].lower() == 'esrisnsr_azimuth']

    if not azimuth_field:
        gnss_metadata_fields['fields'].append({'name': 'esrisnsr_azimuth',
                                                'type': 'esriFieldTypeDouble',
                                                'alias': 'Compass reading (°)',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})

    # esrignss_avg_h_rms
    average_horizontal_accuracy_field = [field for field in feature_layer_fields if
                                        field['name'].lower() == 'esrignss_avg_h_rms']

    if not average_horizontal_accuracy_field:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_avg_h_rms',
                                                'type': 'esriFieldTypeDouble',
                                                'alias': 'Average horizontal accuracy (m)',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})

    # esrignss_avg_v_rms
    average_vertical_accuracy_field = [field for field in feature_layer_fields if
                                    field['name'].lower() == 'esrignss_avg_v_rms']

    if not average_vertical_accuracy_field:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_avg_v_rms',
                                                'type': 'esriFieldTypeDouble',
                                                'alias': 'Average vertical accuracy (m)',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})

    # esrignss_avg_positions
    average_positions_field = [field for field in feature_layer_fields if
                                field['name'].lower() == 'esrignss_avg_positions']

    if not average_positions_field:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_avg_positions',
                                                'type': 'esriFieldTypeInteger',
                                                'alias': 'Averaged positions',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})

    # esrignss_h_stddev"
    standard_deviation_field = [field for field in feature_layer_fields if
                                field['name'].lower() == 'esrignss_h_stddev']

    if not standard_deviation_field:
        gnss_metadata_fields['fields'].append({'name': 'esrignss_h_stddev',
                                                'type': 'esriFieldTypeDouble',
                                                'alias': 'Standard deviation (m)',
                                                'sqlType': 'sqlTypeOther',
                                                'nullable': True,
                                                'editable': True,
                                                'domain': None,
                                                'defaultValue': None})

# Check if AddToDefinition operation needs to be added.
initial_featurelayer_fields_count = len(feature_layer_fields)
if (len(gnss_metadata_fields[
            'fields']) + initial_featurelayer_fields_count
        ) > initial_featurelayer_fields_count:
    operations.append('addToDefinition')

In [ ]:
# Check if any changes are required
if not operations:
    print("No changes to GPS metadata fields.")
else:
    # Add or update service definition
    for operation in operations:

        # Add
        if operation == 'addToDefinition':
            response = feature_layer.manager.add_to_definition(
                gnss_metadata_fields)
            print("Successfully added GPS metadata fields.")

        # Modify
        else:
            response = feature_layer.manager.update_definition(
                feature_layer_fields)
            print("Successfully updated GPS metadata fields.")

        result = response['success']

        if not result:
            print('Failed to update Feature layer service definition.')
        else:
            print('Service definition updated successfully.')  